### Libraries

In [5]:
%%capture
%reset -f  
'generic imports'
import pandas as pd
import json
import time 
import os
import torch
from IPython.display import display, clear_output
from numba import cuda  
from sklearn.model_selection import train_test_split

'data augmentation imports'
from be_great import GReaT # https://github.com/kathrinse/be_great

### GPU

In [7]:
# Select GPU device 
cuda.select_device(0)
# Clear GPU memory
cuda.current_context().deallocations.clear()
# Get the current CUDA context and GPU information
ctx = cuda.current_context()
# Get the current memory information from the cuda context
meminfo = ctx.get_memory_info()
# Get the device name
device_name = cuda.gpus[0].name.decode()
# Get the CUDA version
cudnn_version = cuda.cudadrv.driver.get_version()
if torch.cuda.is_available():
    # dont chang the line
    print('GPU is available', end=' ')
    print(f'({device_name})\nAvailable GPU memory: {meminfo[0] / 1024**3:.1f} GB\ncuDNN version: {cudnn_version}')
else:
    print("No GPU available")

GPU is available (NVIDIA GeForce RTX 4060 Laptop GPU)
Available GPU memory: 7.5 GB
cuDNN version: (12, 2)


### Loading Data

In [21]:
# Load data relative to the training set, with Attack_type "Normal" with 100k rows
df_train = pd.read_csv('../data/EdgeIIot_train_100k.csv', low_memory=False)

#### Calculates the No of rows to be augmented

In [9]:
# Counts how many rows of each Attack_type are missing (100.000)
counts_dict = {}
for attack in df_train["Attack_type"].unique():
    counts_dict[attack] = 100000 - df_train[df_train["Attack_type"] == attack].shape[0]

counts_dict.pop("Normal")

print("Rows to be augmented by attack type:\n")
for key, value in sorted(counts_dict.items(), key=lambda item: item[1], reverse=True):
    print(f"{key:<22} {value:>10}")

Rows to be augmented by attack type:

MITM                        99718
Fingerprinting              99293
Ransomware                  92237
XSS                         87969
Port_Scanning               84085
Backdoor                    80756
Uploading                   70466
DDoS_HTTP                   61084
Password                    60175
DDoS_TCP                    59947
Vulnerability_scanner       59914
SQL_injection               59245
DDoS_ICMP                   45562
DDoS_UDP                     3034


#### Divides data into 2 groups

In [10]:
# Split the data into two groups
counts_dict_1, counts_dict_2 = train_test_split(list(counts_dict.keys()), test_size=0.5, random_state=42)

print("Group 1:", counts_dict_1)
print("Group 2:", counts_dict_2)

print("\nNo of rows by group:")
print("Group 1:", sum([counts_dict[attack] for attack in counts_dict_1]))
print("Group 2:", sum([counts_dict[attack] for attack in counts_dict_2]))

Group 1: ['Password', 'Fingerprinting', 'DDoS_ICMP', 'SQL_injection', 'XSS', 'Backdoor', 'Vulnerability_scanner']
Group 2: ['Uploading', 'Ransomware', 'DDoS_UDP', 'MITM', 'Port_Scanning', 'DDoS_HTTP', 'DDoS_TCP']

No of rows by group:
Group 1: 492914
Group 2: 470571


### Discard Redundant Data

In [11]:
# Since 'Normal' Attack_type already has 100.000 rows, we do not consider it for augmentation
df_attack = df_train[df_train["Attack_type"] != "Normal"]

# Drop the Attack_label because it can be reconstructed from the Attack_type after augmentation
df_attack = df_attack.drop(columns=["Attack_label"])
del df_train

In [7]:
df_attack_1 = df_attack[df_attack["Attack_type"].isin(counts_dict_1)]
# shape of the dataframe before augmentation
print("Shape of the dataframe before augmentation:", df_attack_1.shape)

Shape of the dataframe before augmentation: (207086, 47)


###  Training Data Augmentation Model: Group 1

In [10]:
# instantiate the GReaT model with the distilgpt2 language model
model = GReaT(llm="distilgpt2",                   # Language model to use
              epochs=1,                           # Number of epochs to train
              save_steps=2000,                    # Save model weights every x steps
              logging_steps=500,                  # Log the loss and learning rate every x steps
              experiment_dir="checkpoints/Great/GReat_iter3", # Directory to save the model weights
              batch_size=16                       
             )

In [27]:
# Train the model
model.fit(df_attack_1)

  4%|▍         | 500/12943 [06:45<2:34:27,  1.34it/s] 

{'loss': 0.5297, 'learning_rate': 4.806845399057406e-05, 'epoch': 0.04}


  8%|▊         | 1000/12943 [16:09<2:30:11,  1.33it/s]

{'loss': 0.4146, 'learning_rate': 4.6136907981148113e-05, 'epoch': 0.08}


 12%|█▏        | 1500/12943 [24:54<2:37:32,  1.21it/s] 

...

{'loss': 0.3693, 'learning_rate': 3.6428957737773316e-06, 'epoch': 0.93}


 97%|█████████▋| 12500/12943 [2:59:57<03:25,  2.15it/s]  

{'loss': 0.3687, 'learning_rate': 1.7113497643513869e-06, 'epoch': 0.97}


100%|██████████| 12943/12943 [3:04:00<00:00,  1.17it/s]

{'train_runtime': 11040.1783, 'train_samples_per_second': 18.757, 'train_steps_per_second': 1.172, 'train_loss': 0.38289573543878386, 'epoch': 1.0}


In [28]:
# save the model
model.save("GReat_df_attack_1")
del model, df_attack_1

### Training Data Augmentation Model: Group 2

In [30]:
df_attack_2 = df_attack[df_attack["Attack_type"].isin(counts_dict_2)]
# shape of the dataframe before augmentation
print("Shape of the dataframe before augmentation:", df_attack_2.shape)

Shape of the dataframe before augmentation: (229429, 47)


In [29]:
model2 = GReaT(llm="distilgpt2",                  # Language model to use
              epochs=1,                           # Number of epochs to train
              save_steps=2000,                    # Save model weights every x steps
              logging_steps=500,                  # Log the loss and learning rate every x steps
              experiment_dir="checkpoints/Great/GReat_iter3", # Directory to save the model weights
              batch_size=16                       
             )

In [31]:
model2.fit(df_attack_2)

  3%|▎         | 500/14340 [08:47<1:46:48,  2.16it/s] 

{'loss': 0.5028, 'learning_rate': 4.825662482566248e-05, 'epoch': 0.03}


  7%|▋         | 1000/14340 [18:05<11:01:50,  2.98s/it]

{'loss': 0.3902, 'learning_rate': 4.651324965132497e-05, 'epoch': 0.07}


 10%|█         | 1500/14340 [26:56<3:02:51,  1.17it/s] 

...

 77%|███████▋  | 11000/14340 [3:36:04<1:06:05,  1.19s/it] 

{'loss': 0.3495, 'learning_rate': 1.1645746164574617e-05, 'epoch': 0.77}


 80%|████████  | 11500/14340 [3:50:24<1:22:01,  1.73s/it]

{'loss': 0.3489, 'learning_rate': 9.9023709902371e-06, 'epoch': 0.8}


 81%|████████  | 11588/14340 [3:52:36<41:02,  1.12it/s]  

In [ ]:
# Save the model
model2.save("GReat_df_attack_2")
del df_attack_2

## Data Generation

In [12]:
# Load model df_attack_1
model = GReaT.load_from_dir("checkpoints/GReat/GReat_df_attack_1")

In [13]:
for i, attack in enumerate(counts_dict_1):
    n_att_samples = counts_dict[attack]

    samples = model.sample(n_samples = n_att_samples, 
                           max_length = 1000, 
                           k = 100, 
                           start_col="Attack_type", 
                           start_col_dist={attack: 1}
                           )
    
    if i == 0:
        samples.to_csv(f"EdgeIIoT_train_GReaT_p1.csv", index=False)
    else:
        samples.to_csv(f"EdgeIIoT_train_GReaT_p1.csv", mode='a', header=False, index=False)

60234it [1:45:39,  9.50it/s]                           
99322it [2:57:14,  9.34it/s]                             
45616it [1:19:23,  9.58it/s]                           
59308it [1:49:03,  9.06it/s]                           
87985it [2:35:52,  9.41it/s]                             
80841it [2:22:31,  9.45it/s]                             
100%|██████████| 59914/59914 [1:51:00<00:00,  9.00it/s]


In [15]:
# del df_attack_gen_1
del model
# Load model df_attack_2
model = GReaT.load_from_dir("checkpoints/GReat/GReat_df_attack_2")

In [16]:
for i, attack in enumerate(counts_dict_2):
    n_att_samples = counts_dict[attack]

    samples = model.sample(n_samples = n_att_samples, 
                           max_length = 1000, 
                           k = 128, 
                           start_col="Attack_type", 
                           start_col_dist={attack: 1}
                           )
    
    if i == 0:
        samples.to_csv(f"EdgeIIoT_train_GReaT_p2.csv", index=False)
    else:
        samples.to_csv(f"EdgeIIoT_train_GReaT_p2.csv", mode='a', header=False, index=False)

70486it [1:57:31, 10.00it/s]                           
92361it [2:38:17,  9.72it/s]                             
3065it [05:07,  9.96it/s]                          
99808it [2:49:06,  9.84it/s]                             
84176it [2:23:38,  9.77it/s]                             
61136it [1:41:43, 10.02it/s]                           
59967it [1:42:58,  9.71it/s]                           


## Augmented data verification and concatenation

In [19]:
del df_attack
# load the generated data
df_gen_1 = pd.read_csv("../data/EdgeIIoT_train_GReaT_p1.csv", low_memory=False)
df_gen_2 = pd.read_csv("../data/EdgeIIoT_train_GReaT_p2.csv", low_memory=False)

In [20]:
# print the number of rows by attack type in both generated dataframes
print("df_gen_1:")
print(df_gen_1["Attack_type"].value_counts())
print("\ndf_gen_2:")
print(df_gen_2["Attack_type"].value_counts())


df_gen_1:
Attack_type
Fingerprinting           99293
XSS                      87969
Backdoor                 80756
Password                 60175
Vulnerability_scanner    59914
SQL_injection            59245
DDoS_ICMP                45562
Name: count, dtype: int64

df_gen_2:
Attack_type
MITM             99718
Ransomware       92237
Port_Scanning    84085
Uploading        70466
DDoS_HTTP        61084
DDoS_TCP         59947
DDoS_UDP          3034
Name: count, dtype: int64


In [22]:
# concatenate the generated dataframes
df_gen = pd.concat([df_gen_1, df_gen_2], ignore_index=True)
# concatenate the original dataframe with the generated dataframe
df_train = pd.concat([df_train, df_gen], ignore_index=True)

In [23]:
# count the number of rows by attack type
print("\nTraining set after augmentation:")
print(df_train["Attack_type"].value_counts())


Training set after augmentation:
Attack_type
DDoS_UDP                 100000
Password                 100000
DDoS_TCP                 100000
Backdoor                 100000
DDoS_ICMP                100000
Port_Scanning            100000
Vulnerability_scanner    100000
SQL_injection            100000
DDoS_HTTP                100000
Uploading                100000
XSS                      100000
Ransomware               100000
MITM                     100000
Fingerprinting           100000
Normal                   100000
Name: count, dtype: int64


In [33]:
# list columns of the dataframe
print("\nColumns of the dataframe:")
# don't diplay the index
pd.DataFrame(df_train.columns.values.reshape(-1, 2), columns=["Columns", "of the dataframe"])
# don't diplay the column names
pd.DataFrame(df_train.columns.values.reshape(-1, 2), columns=["", ""])


Columns of the dataframe:


,,
0,arp.opcode,arp.hw.size
1,icmp.checksum,icmp.seq_le
2,icmp.unused,http.content_length
3,http.request.method,http.referer
4,http.request.version,http.response
5,http.tls_port,tcp.ack
6,tcp.ack_raw,tcp.checksum
7,tcp.connection.fin,tcp.connection.rst
8,tcp.connection.syn,tcp.connection.synack
9,tcp.flags,tcp.flags.ack


In [36]:
# df Attack_label reconstruction: if Attack_type is 'Normal' then Attack_label is 0 else 1
df_train["Attack_label"] = df_train["Attack_type"].apply(lambda x: 0 if x == "Normal" else 1)

In [41]:
# check if df_train has any missing values or NaN
print(f"\nMissing values or NaN in the dataframe: {df_train.isnull().values.any()}")


Missing values or NaN in the dataframe: False


In [ ]:
# save the augmented dataframe
df_train.to_csv("../data/EdgeIIoT_train_100k_GReaT.csv", index=False)